In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer

## Import train and test dataset

In [2]:

train_file = "D:\\workspace\\toxic-comments-classification\\dataset\\train.csv";
test_file = "D:\\workspace\\toxic-comments-classification\\dataset\\train.csv";

train_read = pd.read_csv(train_file)
train_read.head()

test_read = pd.read_csv(test_file)
test_read.head()
print(test_read.shape)

(159571, 8)


In [3]:
#reindexing
train_read = train_read.reindex(np.random.permutation(train_read.index))

In [4]:
# seprating comments and labels
comment = train_read['comment_text']
comment.head()
comment = comment.as_matrix()

C:\Users\akshay1\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


In [5]:
label = train_read[['toxic', 'severe_toxic' , 'obscene' , 'threat' , 'insult' , 'identity_hate']]
print(label.head())
label = label.as_matrix()

        toxic  severe_toxic  obscene  threat  insult  identity_hate
144651      0             0        0       0       0              0
53924       0             0        0       0       0              0
63216       0             0        0       0       0              0
44318       0             0        0       0       0              0
89288       0             0        0       0       0              0


C:\Users\akshay1\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


## Data preprocession and visualizing

In [8]:
# Number of comments labelled as toxic(Any type) 

total_comment = len(comment)
total_toxic_comment = 0

for i in range(label.shape[0]):
    count = np.count_nonzero(label[i])
    if count >= 1 :
        total_toxic_comment = total_toxic_comment + 1

print("Total number of Toxic comment")
print(total_toxic_comment)


Total number of Toxic comment
16225


In [9]:
# find average length of comments
lengths_of_all_comments = []

for i in range(total_comment):
    lengths_of_all_comments.append(len(comment[i]))
    
average = sum(lengths_of_all_comments)/total_comment
print('The Average length of Comment: {:.3f}'.format(average))

The Average length of Comment: 394.073


In [10]:
# remove excess length comments
comments = []
labels = []
for i in range(total_comment):
    if(len(comment[i]) <= average):
        comments.append(comment[i])
        labels.append(label[i])
        
print("After filtering comments according to average length ")
print(len(comments))

After filtering comments according to average length 
115232


# Preprocessing

1. Removing stop words
2. removing punctuations
3. stemming and lemmitization
4. Word counts using CountVectorizer
5. Splitting dataset into Training and Testing

In [11]:
#1) removing stop words

#import stop words
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords = stopwords.words('english')

#also the single letter words are of no use and can be removed from the comments. So appending letters from 'b' to 'z'

for x in range(ord('b'), ord('z')+1):
    stopwords.append(chr(x))
    

print(stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\akshay1\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
#2) Preparing punctuations that need to be removed

import string
print(string.punctuation)
punctuation_edit = string.punctuation.replace('\'','') +"0123456789"
print (punctuation_edit)
outtab = "                                         "
trantab = str.maketrans(punctuation_edit, outtab)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
!"#$%&()*+,-./:;<=>?@[\]^_`{|}~0123456789


In [13]:
# initalize stemming and lemminization

lemmatiser = WordNetLemmatizer()
stemmer = PorterStemmer()
#download words from wordnet library
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\akshay1\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
for i in range(len(comments)):
    comments[i] = comments[i].lower().translate(trantab) #remove ounctutations
    outPut = []
    
    for word in comments[i].split():#apply stemming and leminization
        outPut.append(stemmer.stem(lemmatiser.lemmatize(word,pos="v")))
    comments[i] = " ".join(outPut)
               
        